In [2]:
#Start with importing the basic libraries for data cleaning python and numpy
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)

In [3]:
#Read the csv
#You can put r before the  file path, so that python can treat the backslashes as raw and not escape
#Or use double backslashes or use forard slashes
file = r"C:\Users\HP\Desktop\UI\important\Data Science\movie-success-prediction\data\raw\movies_data.csv"
df =  pd.read_csv(file)
df.head()

,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre
0,Superman,tt5950044,2025,7.2,305K,2h 9m,12A,Action
1,One Battle After Another,tt30144839,2025,8.5,15K,2h 41m,15,Action
2,Demon Slayer: Kimetsu no Yaiba Infinity Castle,tt32820897,2025,8.6,37K,2h 35m,15,Action
3,Anaconda,tt33244668,2025,NaN,NaN,NaN,Series,Action
4,KPop Demon Hunters,tt14205554,2025,7.6,79K,1h 35m,PG,Action


In [4]:
print(len(df))

34892


In [5]:
#check if there are duplicates, data was scraped from imdb and some might have been repeated.
df.duplicated(subset = "name").sum()
df= df.drop_duplicates(subset = "name")
df.duplicated(subset = "imdb_id").sum()

np.int64(0)

In [6]:
#Check for null and drop them
df.isna().any(axis=1).sum()
df = df.dropna(axis=0)
print(df.isna().any(axis=0))
print(df.shape)


name                   False
imdb_id                False
year                   False
ratings                False
vote_count             False
movie_duration         False
movie_certification    False
genre                  False
dtype: bool
(17894, 8)


In [7]:
df.head()

,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre
0,Superman,tt5950044,2025,7.2,305K,2h 9m,12A,Action
1,One Battle After Another,tt30144839,2025,8.5,15K,2h 41m,15,Action
2,Demon Slayer: Kimetsu no Yaiba Infinity Castle,tt32820897,2025,8.6,37K,2h 35m,15,Action
4,KPop Demon Hunters,tt14205554,2025,7.6,79K,1h 35m,PG,Action
5,F1,tt16311594,2025,7.8,214K,2h 35m,12A,Action


In [8]:
#Checking for the unique values in some selected columns
print(df["year"].unique())
print(df["ratings"].unique())
print(df["movie_certification"].unique())
print(df["genre"].unique())

['2025' '2024' '2008' '2010' '2021' '2023' '2022' '1999' '1995' '2019'
 '2001' '2003' '2000' '2005' '2014' '2012' '2013' '1993' '1994' '1989'
 '2017' '2020' '1997' '2002' '1986' '2015' '1991' '2018' '1977' '1988'
 '2016' '1982' '2004' '2009' '2011' '2007' '1984' '1992' '1980' '1954'
 '1981' '1996' '1987' '1998' '2006' '1990' '1983' '1972' '1985' '1971'
 '1979' '1962' '1973' '1964' '1967' '1976' '1963' '1961' '1965' '1968'
 '1969' '1924' '1970' '1974' '1926' '1978' '1966' '1975' '1939' '1959'
 '1957' '1960' '1956' '1948' '1937' '1955' '1953' '1951' '1944' '1925'
 '1942' '1958' '1935' '1938' '1940' '1949' '1946' '1947' '1941' '1945'
 '1928' '1952' '1943' '1934' '1936' '1933' '1930' '1931' '1950' '1922'
 '1929' '1932' '1921' '1916' '1914' '1919' '1920' '1927' '1897' '1899'
 '1900' '1906' '1917' '1918' '1923']
[7.2 8.5 8.6 7.6 7.8 7.  6.5 8.7 6.3 4.8 5.8 6.7 5.9 6.9 8.9 8.1 9.1 8.8
 7.4 5.7 4.5 7.1 7.3 2.4 6.1 8.2 5.6 8.3 8.4 5.3 6.  6.6 6.8 8.  5.5 6.4
 7.5 4.4 7.7 6.2 7.9 5.1 5.4 4.7 4.1

In [9]:
#Checking the length of the documents
print(len(df["year"].unique()))
print(len(df["ratings"].unique()))
print(len(df["movie_certification"].unique()))
print(len(df["genre"].unique()))


115
90
100
26


In [10]:
#defining a function to clean the votecount
def clean_vote(x):
    if isinstance(x, str):
        x = x.lower().strip()
        if "k" in x:
            return int(float(x.replace("k", "")) *1000)
        elif "m" in x:
            return int(float(x.replace("m", "")) *1_000_000)
        else:
            return int(float(x))
    else:
        return x

df["vote_count"] = df["vote_count"].apply(clean_vote)
df.head()



,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre
0,Superman,tt5950044,2025,7.2,305000,2h 9m,12A,Action
1,One Battle After Another,tt30144839,2025,8.5,15000,2h 41m,15,Action
2,Demon Slayer: Kimetsu no Yaiba Infinity Castle,tt32820897,2025,8.6,37000,2h 35m,15,Action
4,KPop Demon Hunters,tt14205554,2025,7.6,79000,1h 35m,PG,Action
5,F1,tt16311594,2025,7.8,214000,2h 35m,12A,Action


In [11]:
#to convert duration to minutes alone
import re

def clean_duration(x):
    if isinstance(x, str):
        hours = 0
        minutes = 0
        
        # Find hours
        h_match = re.search(r"(\d+)\s*h", x.lower())
        if h_match:
            hours = int(h_match.group(1))
        
        # Find minutes
        m_match = re.search(r"(\d+)\s*m", x.lower())
        if m_match:
            minutes = int(m_match.group(1))
        
        return hours * 60 + minutes
    else:
        return x
    
df["movie_duration"] = df["movie_duration"].apply(clean_duration)
df.head()


,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre
0,Superman,tt5950044,2025,7.2,305000,129,12A,Action
1,One Battle After Another,tt30144839,2025,8.5,15000,161,15,Action
2,Demon Slayer: Kimetsu no Yaiba Infinity Castle,tt32820897,2025,8.6,37000,155,15,Action
4,KPop Demon Hunters,tt14205554,2025,7.6,79000,95,PG,Action
5,F1,tt16311594,2025,7.8,214000,155,12A,Action


In [ ]:
#Basic stats across numeric columns
df.describe()

,ratings,vote_count,movie_duration
count,17894.000000,1.789400e+04,17894.000000
mean,6.549011,5.481271e+04,107.522298
std,1.060663,1.497696e+05,386.480769
min,1.000000,5.000000e+00,0.000000
25%,6.000000,1.100000e+03,90.000000
50%,6.600000,6.300000e+03,100.000000
75%,7.300000,4.100000e+04,115.000000
max,9.900000,3.100000e+06,51420.000000


In [15]:
#I want to check for the column with the minimum value of movie duration, because a movie with min value of zero doesn't make sense
min_duration = df[df["movie_duration"] == df["movie_duration"].min()]
min_duration

,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre
219,Der Tiger,tt31495504,2025,7.6,114,0,Series,Action
10384,OnlyFans: Selling Sexy,tt14107670,2021,4.8,184,0,Series,Documentary
20982,Indian 2,tt8066940,2024,3.8,17000,0,Series,Musical
30145,Life of the Party,tt0382717,2005,5.3,346,0,Series,Sport
34815,Cheyenne,tt0219588,1996,4.8,237,0,Series,Western


In [ ]:
#Checking to confirm it was what was on the site that was scrapped and not a cleaning error
#Shows it is an error while scraping, checked the site too and some of the movies had issues like that
path1 = r"C:\Users\HP\Desktop\UI\important\Data Science\movie-success-prediction\data\raw\movies_data.csv"
df1 = pd.read_csv(path1)
df1[df1["name"].isin(min_duration["name"])]

,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre
219,Der Tiger,tt31495504,2025,7.6,114,R,Series,Action
10384,OnlyFans: Selling Sexy,tt14107670,2021,4.8,184,TV-MA,Series,Documentary
10891,Der Tiger,tt31495504,2025,7.6,114,R,Series,Drama
20982,Indian 2,tt8066940,2024,3.8,17K,15,Series,Musical
30145,Life of the Party,tt0382717,2005,5.3,346,R,Series,Sport
31927,Der Tiger,tt31495504,2025,7.6,114,R,Series,War
34815,Cheyenne,tt0219588,1996,4.8,237,R,Series,Western


In [19]:
#drop rows with no movie_duration
df = df.drop(min_duration.index)
df[df["name"].isin(min_duration["name"])]

,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre


In [21]:
#metascore is not a movie certification, so check for it
metascore = df[df["movie_certification"].str.contains("Metascore", na=False)][["name", "movie_duration", "movie_certification"]]
metascore.head()


,name,movie_duration,movie_certification
53,Sirât,115,82Metascore
208,Marching Powder,96,39Metascore
391,Sacrifice,103,48Metascore
510,Fuze,98,58Metascore
1059,Eagles of the Republic,129,76Metascore


In [ ]:
# count rows where certificate contains "Metascore"
metascore_count = df[df["movie_certification"].str.contains("Metascore", na=False)].shape[0]

print("Number of movies with Metascore:", metascore_count)


Number of movies with Metascore: 242


In [22]:

#Drop metascore as it is not much
df = df.drop(metascore.index)
df[df["name"].isin(metascore["name"])]

,name,imdb_id,year,ratings,vote_count,movie_duration,movie_certification,genre


In [23]:
df.dtypes

name                    object
imdb_id                 object
year                     Int64
ratings                float64
vote_count               int64
movie_duration           int64
movie_certification     object
genre                   object
dtype: object

In [24]:
#converting year into integer
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype("Int64")
df.to_csv(r"C:\Users\HP\Desktop\UI\important\Data Science\movie-success-prediction\data\processed\cleaned_movies_data.csv", index=False)